In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#@author sourabhwarrier 2022
#   IMPORTS BEGIN
import pickle
import numpy as np
import cv2
import pandas as pd
from random import random, shuffle
#import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model, utils, optimizers
import tensorflow as tf
#   IMPORTS END

#   PREPROCESSOR STAGE BEGIN
dims = (128,128)
color = 1
mtype = "CNN"
colorflag = "color" if color else "gray"
def imgage_tensor(filepath,dims):
    if colorflag == "gray":   
        image = cv2.cvtColor(cv2.resize(cv2.imread(filepath),dims),cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.resize(cv2.imread(filepath),dims)
    img_data = np.array(image)    
    return img_data

#def build_train_tensor(dims,ftype):
#    meta = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/{}.csv".format(ftype))
#    data_tensor = []
#    size = meta.shape[0]
#    for i in range(size):
#        try:
#            img_data = imgage_tensor("/kaggle/input/petfinder-pawpularity-score/{}/{}.jpg".format(ftype,meta.iloc[i][0]),dims = dims)
#            pawpularity = float(meta.iloc[i][13])
#            #metadata = np.array([meta.iloc[i][1],meta.iloc[i][2],meta.iloc[i][3],meta.iloc[i][4],meta.iloc[i][5],meta.iloc[i][6],meta.iloc[i][7],meta.iloc[i][8],meta.iloc[i][9],meta.iloc[i][10],meta.iloc[i][11],meta.iloc[i][12]])
#            data_tensor.append((img_data/255,pawpularity/100))
#            if i%100 == 1:
#               print("processed images : {}".format(i-1))
#        except:
#            pass
#    shuffle(data_tensor)
#    return data_tensor

def build_test_tensor(dims,ftype):
    meta = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/{}.csv".format(ftype))
    data_tensor = []
    size = meta.shape[0]
    for i in range(size):
        try:
            Id = str(meta.iloc[i][0])
            img_data = imgage_tensor("/kaggle/input/petfinder-pawpularity-score/{}/{}.jpg".format(ftype,meta.iloc[i][0]),dims = dims)
            data_tensor.append((Id,img_data/255))
            if i%100 == 1:
                print("processed images : {}".format(i-1))
        except:
            pass
    return data_tensor
try:
    #with open('{}_train_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'rb') as handle:
    #    train_tensor = pickle.load(handle)
    with open('{}_test_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'rb') as handle:
        test_tensor = pickle.load(handle)
    #with open('{}_final_test_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'rb') as handle:
    #    final_test_tensor = pickle.load(handle)
    print("tensors found : will skip preprocessing stage")
except FileNotFoundError:
    print("tensors not found : starting preprocessing")
    #data_tensor = build_train_tensor(dims,"train")
    test_tensor = build_test_tensor(dims,"test")
    #train_tensor,final_test_tensor = data_tensor[:7212],data_tensor[7212:]
    #with open('{}_train_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'wb') as handle:
    #    pickle.dump(train_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}_test_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'wb') as handle:
        pickle.dump(test_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #with open('{}_final_test_tensor_{}.pickle'.format(colorflag,str(dims[0])), 'wb') as handle:
    #    pickle.dump(final_test_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)
#   PREPROCESSOR STAGE END

#   MODEL TRAIN BEGIN
try:
    if mtype == "DNN":
        model = tf.keras.models.load_model('dnmodel{}{}.h5'.format(colorflag,dims[0]))
    else:
        model = tf.keras.models.load_model('/kaggle/input/pawpularity-cnn/cnnmodel{}{}.h5'.format(colorflag,dims[0]))
    print("model found : will skip to predictions")
except:
    print("model not found : will train first and then predict")
    initializer = tf.keras.initializers.GlorotNormal()
    if mtype == "DNN":
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(dims[0], dims[0],3)),
            #tf.keras.layers.Dense(2048, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(256, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(128, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(64, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(64, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(32, activation='relu',kernel_initializer=initializer),
            tf.keras.layers.Dense(1)
        ])
    else:
        model = tf.keras.Sequential()
        #model.add(Dense(256, activation='relu', input_dim=366))
        model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape = (dims[0], dims[0],3)))
        #model.add(Conv2D(128, (3, 3), activation='relu'))
        #model.add(Conv2D(64, (3, 3), init='uniform'))

        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Flatten())

        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.1))

        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.1))
        model.add(tf.keras.layers.Dense(32, activation='relu'))

        model.add(tf.keras.layers.Dense(1, activation='linear'))

    x_train,y_train = np.array([x[0] for x in train_tensor]),np.array([x[1] for x in train_tensor])
    x_final_test,y_final_test = np.array([x[0] for x in final_test_tensor]),np.array([x[1] for x in final_test_tensor])
    opt = optimizers.Adam(learning_rate=0.000001)
    model.compile(optimizer = opt, loss = 'MSE',metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.fit(x_train, y_train, epochs = 50,validation_split = 0.15)
    if mtype == "DNN":
        model.save('dnnmodel{}{}.h5'.format(colorflag,dims[0]))
    else:
        model.save('cnnmodel{}{}.h5'.format(colorflag,dims[0]))
#   MODEL TRAIN END

#   PREDICTIONS BEGIN
#model.evaluate(x_final_test,y_final_test)
Ids,x_test = np.array([x[0] for x in test_tensor]),np.array([x[1] for x in test_tensor])
predictions = [round(float(x[0])*100,2) for x in model.predict(x_test)]
submission = pd.DataFrame({'Id':Ids,'Pawpularity':predictions})
submission.to_csv("submission.csv",index=False)
print(submission)
#   PREDICTIONS END


tensors not found : starting preprocessing
processed images : 0


2022-01-06 18:20:15.485317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 18:20:15.587122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 18:20:15.587863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 18:20:15.590143: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

model found : will skip to predictions


2022-01-06 18:20:18.631159: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-06 18:20:19.564978: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


                                 Id  Pawpularity
0  4128bae22183829d2b5fea10effdb0c3        37.69
1  43a2262d7738e3d420d453815151079e        35.89
2  4e429cead1848a298432a0acad014c9d        38.38
3  80bc3ccafcc51b66303c2c263aa38486        38.87
4  8f49844c382931444e68dffbe20228f4        38.52
5  b03f7041962238a7c9d6537e22f9b017        39.01
6  c978013571258ed6d4637f6e8cc9d6a3        38.56
7  e0de453c1bffc20c22b072b34b54e50f        38.17
